Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
import math

First reload the data we generated in _notmist.ipynb_.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 2 to [0.0, 1.0, 0.0 ...], 3 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

### Logistic network

In [6]:
batch_size = 128
regularization_beta = 5e-4

graph = tf.Graph()
with graph.as_default():

    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    # Variables.
    weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
    biases = tf.Variable(tf.zeros([num_labels]))

    # Training computation.
    logits = tf.matmul(tf_train_dataset, weights) + biases
    loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits = logits, labels = tf_train_labels))

    # L2 Regularization
    
    regularization = (tf.nn.l2_loss(weights) + tf.nn.l2_loss(biases))
    
    loss = loss + regularization_beta * regularization

    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
    test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [7]:
num_steps = 1001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 100 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Instructions for updating:
Use `tf.global_variables_initializer` instead.
Initialized
Minibatch loss at step 0: 21.389757
Minibatch accuracy: 7.0%
Validation accuracy: 10.2%
Minibatch loss at step 100: 3.691218
Minibatch accuracy: 69.5%
Validation accuracy: 70.8%
Minibatch loss at step 200: 3.278759
Minibatch accuracy: 75.0%
Validation accuracy: 73.7%
Minibatch loss at step 300: 2.298655
Minibatch accuracy: 82.0%
Validation accuracy: 74.3%
Minibatch loss at step 400: 2.660796
Minibatch accuracy: 73.4%
Validation accuracy: 73.9%
Minibatch loss at step 500: 2.356946
Minibatch accuracy: 77.3%
Validation accuracy: 75.4%
Minibatch loss at step 600: 2.157014
Minibatch accuracy: 75.8%
Validation accuracy: 76.0%
Minibatch loss at step 700: 1.944165
Minibatch accuracy: 81.2%
Validation accuracy: 75.9%
Minibatch loss at step 800: 2.466915
Minibatch accuracy: 73.4%
Validation accuracy: 76.7%
Minibatch loss at step 900: 1.919496
Minibatch accuracy: 79.7%
Validation accuracy: 76.2%
Minibatch loss a

In [8]:
Neural network

SyntaxError: invalid syntax (<ipython-input-8-20065e2173dd>, line 1)

In [9]:
batch_size = 128
hidden_layer_size = 1024

graph = tf.Graph()
with graph.as_default():

    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    # Variables.
    
    # Hidden layer (RELU magic)
    
    weights_hidden = tf.Variable(
        tf.truncated_normal([image_size * image_size, hidden_layer_size]))
    biases_hidden = tf.Variable(tf.zeros([hidden_layer_size]))
    hidden = tf.nn.relu(tf.matmul(tf_train_dataset, weights_hidden) + biases_hidden)

    # Output layer

    weights_output = tf.Variable(
        tf.truncated_normal([hidden_layer_size, num_labels]))
    biases_output = tf.Variable(tf.zeros([num_labels]))
    
    # Training computation.
    
    logits = tf.matmul(hidden, weights_output) + biases_output
    
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(logits = logits, labels = tf_train_labels))
  
    # L2 regularization on hidden and output weights and biases
    
    regularizers = (tf.nn.l2_loss(weights_hidden) + tf.nn.l2_loss(biases_hidden) + 
                   tf.nn.l2_loss(weights_output) + tf.nn.l2_loss(biases_output))

    loss = loss + 5e-4 * regularizers
    
    # Optimizer.

    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
    # Predictions for the training, validation, and test data.

    # Creation of hidden layer of RELU for the validation and testing process
    
    
    train_prediction = tf.nn.softmax(logits)
    
    hidden_validation = tf.nn.relu(tf.matmul(tf_valid_dataset, weights_hidden) + biases_hidden)
    valid_prediction = tf.nn.softmax(
    tf.matmul(hidden_validation, weights_output) + biases_output)
    
    hidden_prediction = tf.nn.relu(tf.matmul(tf_test_dataset, weights_hidden) + biases_hidden)
    test_prediction = tf.nn.softmax(tf.matmul(hidden_prediction, weights_output) + biases_output)

In [10]:
num_steps = 1001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 100 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Instructions for updating:
Use `tf.global_variables_initializer` instead.
Initialized
Minibatch loss at step 0: 493.764557
Minibatch accuracy: 18.8%
Validation accuracy: 20.2%
Minibatch loss at step 100: 184.306824
Minibatch accuracy: 77.3%
Validation accuracy: 74.4%
Minibatch loss at step 200: 170.814194
Minibatch accuracy: 82.8%
Validation accuracy: 79.3%
Minibatch loss at step 300: 139.608490
Minibatch accuracy: 83.6%
Validation accuracy: 81.0%
Minibatch loss at step 400: 142.290268
Minibatch accuracy: 80.5%
Validation accuracy: 77.8%
Minibatch loss at step 500: 131.369736
Minibatch accuracy: 82.0%
Validation accuracy: 80.9%
Minibatch loss at step 600: 123.884476
Minibatch accuracy: 74.2%
Validation accuracy: 77.2%
Minibatch loss at step 700: 115.832916
Minibatch accuracy: 84.4%
Validation accuracy: 80.1%
Minibatch loss at step 800: 109.707718
Minibatch accuracy: 80.5%
Validation accuracy: 81.2%
Minibatch loss at step 900: 102.423866
Minibatch accuracy: 83.6%
Validation accuracy: 80

---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

### Logistic network

In [13]:
batch_size = 15
regularization_beta = 5e-4

graph = tf.Graph()
with graph.as_default():

    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    # Variables.
    weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
    biases = tf.Variable(tf.zeros([num_labels]))

    # Training computation.
    logits = tf.matmul(tf_train_dataset, weights) + biases
    loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits = logits, labels = tf_train_labels))

    # L2 Regularization
    
    regularization = (tf.nn.l2_loss(weights) + tf.nn.l2_loss(biases))
    
    loss = loss + regularization_beta * regularization

    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
    test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [14]:
num_steps = 1001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 100 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Instructions for updating:
Use `tf.global_variables_initializer` instead.
Initialized
Minibatch loss at step 0: 17.874863
Minibatch accuracy: 13.3%
Validation accuracy: 15.2%
Minibatch loss at step 100: 4.756828
Minibatch accuracy: 80.0%
Validation accuracy: 68.7%
Minibatch loss at step 200: 2.602775
Minibatch accuracy: 93.3%
Validation accuracy: 73.0%
Minibatch loss at step 300: 2.018363
Minibatch accuracy: 80.0%
Validation accuracy: 72.9%
Minibatch loss at step 400: 3.294932
Minibatch accuracy: 73.3%
Validation accuracy: 73.3%
Minibatch loss at step 500: 2.195554
Minibatch accuracy: 80.0%
Validation accuracy: 74.9%
Minibatch loss at step 600: 4.420096
Minibatch accuracy: 60.0%
Validation accuracy: 73.2%
Minibatch loss at step 700: 3.290654
Minibatch accuracy: 73.3%
Validation accuracy: 71.3%
Minibatch loss at step 800: 2.689300
Minibatch accuracy: 60.0%
Validation accuracy: 71.4%
Minibatch loss at step 900: 2.185731
Minibatch accuracy: 80.0%
Validation accuracy: 74.0%
Minibatch loss 

### Neural Network

In [15]:
batch_size = 10
hidden_layer_size = 1024
regularization_beta = 5e-4

graph = tf.Graph()
with graph.as_default():

    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    # Variables.
    
    # Hidden layer (RELU magic)
    
    weights_hidden = tf.Variable(
        tf.truncated_normal([image_size * image_size, hidden_layer_size], 
                           stddev=1 / math.sqrt(float(image_size * image_size))))
    biases_hidden = tf.Variable(tf.zeros([hidden_layer_size]))
    hidden = tf.nn.relu(tf.matmul(tf_train_dataset, weights_hidden) + biases_hidden)

    # Output layer

    weights_output = tf.Variable(
        tf.truncated_normal([hidden_layer_size, num_labels]))
    biases_output = tf.Variable(tf.zeros([num_labels]))
    
    # Training computation.
    
    logits = tf.matmul(hidden, weights_output) + biases_output
    
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(logits = logits,labels = tf_train_labels))
  
    # L2 regularization on hidden and output weights and biases
    
    regularizers = (tf.nn.l2_loss(weights_hidden) + tf.nn.l2_loss(biases_hidden) + 
                   tf.nn.l2_loss(weights_output) + tf.nn.l2_loss(biases_output))

    loss = loss + regularization_beta * regularizers
    
    # Optimizer.

    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
    # Predictions for the training, validation, and test data.

    # Creation of hidden layer of RELU for the validation and testing process
    
    
    train_prediction = tf.nn.softmax(logits)
    
    hidden_validation = tf.nn.relu(tf.matmul(tf_valid_dataset, weights_hidden) + biases_hidden)
    valid_prediction = tf.nn.softmax(
    tf.matmul(hidden_validation, weights_output) + biases_output)
    
    hidden_prediction = tf.nn.relu(tf.matmul(tf_test_dataset, weights_hidden) + biases_hidden)
    test_prediction = tf.nn.softmax(tf.matmul(hidden_prediction, weights_output) + biases_output)

In [16]:
num_steps = 1001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 100 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Instructions for updating:
Use `tf.global_variables_initializer` instead.
Initialized
Minibatch loss at step 0: 14.091565
Minibatch accuracy: 0.0%
Validation accuracy: 19.2%
Minibatch loss at step 100: 109.809464
Minibatch accuracy: 80.0%
Validation accuracy: 55.1%
Minibatch loss at step 200: 1739.378906
Minibatch accuracy: 40.0%
Validation accuracy: 41.0%
Minibatch loss at step 300: 5911.374023
Minibatch accuracy: 10.0%
Validation accuracy: 45.4%
Minibatch loss at step 400: 92240.820312
Minibatch accuracy: 20.0%
Validation accuracy: 40.8%
Minibatch loss at step 500: 38677.519531
Minibatch accuracy: 40.0%
Validation accuracy: 56.2%
Minibatch loss at step 600: 229623.734375
Minibatch accuracy: 50.0%
Validation accuracy: 42.6%
Minibatch loss at step 700: 31457.800781
Minibatch accuracy: 40.0%
Validation accuracy: 50.8%
Minibatch loss at step 800: 23581.250000
Minibatch accuracy: 60.0%
Validation accuracy: 54.8%
Minibatch loss at step 900: 49345.230469
Minibatch accuracy: 40.0%
Validation

---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [17]:
batch_size = 20
hidden_layer_size = 2048 # Doubled because half of the results are discarded
regularization_beta = 5e-4

graph = tf.Graph()
with graph.as_default():

    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    # Variables.
    
    # Hidden layer (RELU magic)
    
    weights_hidden = tf.Variable(
        tf.truncated_normal([image_size * image_size, hidden_layer_size], 
                           stddev=1 / math.sqrt(float(image_size * image_size))))
    biases_hidden = tf.Variable(tf.zeros([hidden_layer_size]))
    hidden = tf.nn.relu(tf.matmul(tf_train_dataset, weights_hidden) + biases_hidden)

    # Output layer

    weights_output = tf.Variable(
        tf.truncated_normal([hidden_layer_size, num_labels]))
    biases_output = tf.Variable(tf.zeros([num_labels]))
    
    # Training computation.
    
    logits = tf.matmul(tf.nn.dropout(hidden, 0.5), weights_output) + biases_output
    
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(logits = logits,labels = tf_train_labels))
  
    # L2 regularization on hidden and output weights and biases
    
    regularizers = (tf.nn.l2_loss(weights_hidden) + tf.nn.l2_loss(biases_hidden) + 
                   tf.nn.l2_loss(weights_output) + tf.nn.l2_loss(biases_output))

    loss = loss + regularization_beta * regularizers
    
    # Optimizer.

    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
    # Predictions for the training, validation, and test data.

    # Creation of hidden layer of RELU for the validation and testing process
    
    
    train_prediction = tf.nn.softmax(logits)
    
    hidden_validation = tf.nn.relu(tf.matmul(tf_valid_dataset, weights_hidden) + biases_hidden)
    valid_prediction = tf.nn.softmax(
    tf.matmul(hidden_validation, weights_output) + biases_output)
    
    hidden_prediction = tf.nn.relu(tf.matmul(tf_test_dataset, weights_hidden) + biases_hidden)
    test_prediction = tf.nn.softmax(tf.matmul(hidden_prediction, weights_output) + biases_output)

In [18]:
num_steps = 1001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 100 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Instructions for updating:
Use `tf.global_variables_initializer` instead.
Initialized
Minibatch loss at step 0: 32.112591
Minibatch accuracy: 15.0%
Validation accuracy: 25.9%
Minibatch loss at step 100: 703686.250000
Minibatch accuracy: 30.0%
Validation accuracy: 33.7%
Minibatch loss at step 200: 42167440.000000
Minibatch accuracy: 25.0%
Validation accuracy: 37.7%
Minibatch loss at step 300: 8292377088.000000
Minibatch accuracy: 20.0%
Validation accuracy: 42.3%
Minibatch loss at step 400: 625634902016.000000
Minibatch accuracy: 45.0%
Validation accuracy: 51.2%
Minibatch loss at step 500: 15849037496320.000000
Minibatch accuracy: 45.0%
Validation accuracy: 40.5%
Minibatch loss at step 600: 320723025395712.000000
Minibatch accuracy: 45.0%
Validation accuracy: 46.6%
Minibatch loss at step 700: 56503563248992256.000000
Minibatch accuracy: 40.0%
Validation accuracy: 49.8%
Minibatch loss at step 800: 4569175875240591360.000000
Minibatch accuracy: 45.0%
Validation accuracy: 46.9%
Minibatch lo

---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [ ]:
batch_size = 128
hidden_layer_size = 2048 # Doubled because half of the results are discarded
regularization_beta = 5e-4

graph = tf.Graph()
with graph.as_default():

    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    # Variables.
    
    # Hidden layer (RELU magic)
    
    weights_hidden_1 = tf.Variable(
        tf.truncated_normal([image_size * image_size, hidden_layer_size], 
                           stddev=1 / math.sqrt(float(image_size * image_size))))
    biases_hidden_1 = tf.Variable(tf.zeros([hidden_layer_size]))
    hidden_1 = tf.nn.relu(tf.matmul(tf_train_dataset, weights_hidden_1) + biases_hidden_1)

    weights_hidden_2 = tf.Variable(tf.truncated_normal([hidden_layer_size, hidden_layer_size],
                                                      stddev=1 / math.sqrt(float(image_size * image_size))))
    biases_hidden_2 = tf.Variable(tf.zeros([hidden_layer_size]))
    hidden_2 = tf.nn.relu(tf.matmul(tf.nn.dropout(hidden_1, 0.5), weights_hidden_2) + biases_hidden_2)
    
    # Output layer

    weights_output = tf.Variable(
        tf.truncated_normal([hidden_layer_size, num_labels],
                           stddev=1 / math.sqrt(float(image_size * image_size))))
    biases_output = tf.Variable(tf.zeros([num_labels]))
    
    # Training computation.
    
    logits = tf.matmul(tf.nn.dropout(hidden_2, 0.5), weights_output) + biases_output
    
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(logits = logits,labels = tf_train_labels))
  
    # L2 regularization on hidden and output weights and biases
    
    regularizers = (tf.nn.l2_loss(weights_hidden_1) + tf.nn.l2_loss(biases_hidden_1) + 
                    tf.nn.l2_loss(weights_hidden_2) + tf.nn.l2_loss(biases_hidden_2) +
                    tf.nn.l2_loss(weights_output) + tf.nn.l2_loss(biases_output))

    loss = loss + regularization_beta * regularizers
    
    # Optimizer.

    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
    # Predictions for the training, validation, and test data.

    # Creation of hidden layer of RELU for the validation and testing process
    
    
    train_prediction = tf.nn.softmax(logits)
    
    hidden_validation_1 = tf.nn.relu(tf.matmul(tf_valid_dataset, weights_hidden_1) + biases_hidden_1)
    hidden_validation_2 = tf.nn.relu(tf.matmul(hidden_validation_1, weights_hidden_2) + biases_hidden_2)
    valid_prediction = tf.nn.softmax(
    tf.matmul(hidden_validation_2, weights_output) + biases_output)
    
    hidden_prediction_1 = tf.nn.relu(tf.matmul(tf_test_dataset, weights_hidden_1) + biases_hidden_1)
    hidden_prediction_2 = tf.nn.relu(tf.matmul(hidden_prediction_1, weights_hidden_2) + biases_hidden_2)
    test_prediction = tf.nn.softmax(tf.matmul(hidden_prediction_2, weights_output) + biases_output)

In [ ]:
num_steps = 10001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Instructions for updating:
Use `tf.global_variables_initializer` instead.
Initialized
Minibatch loss at step 0: 4.049647
Minibatch accuracy: 9.4%
Validation accuracy: 35.5%
Minibatch loss at step 500: 1.688743
Minibatch accuracy: 84.4%
Validation accuracy: 84.2%
